In [2]:
import torch
from chadhmm import GaussianHMM, MultinomialHMM, PoissonHMM, GaussianMixtureHMM
from chadhmm import Transitions, CovarianceType

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")


ModuleNotFoundError: No module named 'chadhmm'

## 1. Create a GaussianHMM Model on CPU (Default)


In [ ]:
# Create a Gaussian HMM model
n_states = 3
n_features = 2

hmm = GaussianHMM(
    n_states=n_states,
    n_features=n_features,
    transitions=Transitions.ERGODIC,
    covariance_type=CovarianceType.FULL,
    alpha=1.0,
    seed=42
)

# Generate some test data
torch.manual_seed(42)
X = torch.randn(100, n_features)
lengths = [50, 50]

print(f"Model created on device: {next(hmm.parameters()).device}")
print(f"Data is on device: {X.device}")
print(f"\nModel parameters devices:")
for name, param in hmm.named_parameters():
    print(f"  {name}: {param.device}")


## 2. Move Model to Different Devices

Since HMM models inherit from `nn.Module`, you can use PyTorch's standard `.to()` method to move them between devices.


In [ ]:
# Check available devices and select one
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_name = "CUDA GPU"
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    device_name = "MPS (Apple Silicon)"
else:
    device = torch.device("cpu")
    device_name = "CPU"

print(f"Moving model to: {device_name}")

# Move model to the selected device
hmm.to(device)

print(f"\nModel is now on device: {next(hmm.parameters()).device}")
print(f"\nAll model parameters after moving:")
for name, param in hmm.named_parameters():
    print(f"  {name}: {param.device}, shape: {param.shape}")


## 3. Train Model on the Target Device

When training, make sure your data is also on the same device as the model.


In [ ]:
# Move data to the same device as the model
X_device = X.to(device)

print(f"Data moved to device: {X_device.device}")

# Train the model
hmm.fit(
    X=X_device,
    lengths=lengths,
    max_iter=10,
    n_init=1,
    verbose=False
)

print(f"\nModel trained successfully on {device_name}!")
print(f"Final log-likelihood: {hmm.score(X_device, lengths=lengths, by_sample=False).item():.2f}")


## 4. Move Model Back to CPU

After training on GPU, you might want to move the model back to CPU for inference or saving.


In [ ]:
# Move model back to CPU
hmm.to('cpu')

print(f"Model moved back to: {next(hmm.parameters()).device}")

# Move data back to CPU for inference
X_cpu = X_device.to('cpu')

# Perform inference on CPU
predictions = hmm.predict(X_cpu, lengths=lengths)
print(f"\nPredictions shape: {len(predictions)} sequences")
print(f"First sequence predictions (first 10 states): {predictions[0][:10].tolist()}")


## 5. Testing with Different HMM Types

Let's test device movement with other HMM types to ensure consistency.


In [ ]:
# Test with MultinomialHMM
print("=" * 60)
print("Testing MultinomialHMM")
print("=" * 60)

multinomial_hmm = MultinomialHMM(
    n_states=3,
    n_features=4,
    n_trials=10,
    transitions=Transitions.ERGODIC,
    seed=42
)

# Generate multinomial data
X_mult = torch.randint(0, 4, (100,))
X_mult_onehot = 10 * torch.nn.functional.one_hot(X_mult, 4)

print(f"Initial device: {next(multinomial_hmm.parameters()).device}")

# Move to target device
multinomial_hmm.to(device)
X_mult_device = X_mult_onehot.to(device)

print(f"After .to({device}): {next(multinomial_hmm.parameters()).device}")

# Quick fit
multinomial_hmm.fit(X_mult_device, lengths=[50, 50], max_iter=5, n_init=1, verbose=False)
print(f"Training completed on {device}")


In [ ]:
# Test with PoissonHMM
print("\n" + "=" * 60)
print("Testing PoissonHMM")
print("=" * 60)

poisson_hmm = PoissonHMM(
    n_states=3,
    n_features=2,
    transitions=Transitions.ERGODIC,
    seed=42
)

# Generate Poisson data
X_poisson = torch.poisson(torch.ones(100, 2) * 3)

print(f"Initial device: {next(poisson_hmm.parameters()).device}")

# Move to target device
poisson_hmm.to(device)
X_poisson_device = X_poisson.to(device)

print(f"After .to({device}): {next(poisson_hmm.parameters()).device}")

# Quick fit
poisson_hmm.fit(X_poisson_device, lengths=[50, 50], max_iter=5, n_init=1, verbose=False)
print(f"Training completed on {device}")


In [ ]:
# Test with GaussianMixtureHMM
print("\n" + "=" * 60)
print("Testing GaussianMixtureHMM")
print("=" * 60)

gmm_hmm = GaussianMixtureHMM(
    n_states=2,
    n_features=2,
    n_components=3,
    transitions=Transitions.ERGODIC,
    covariance_type=CovarianceType.FULL,
    seed=42
)

# Generate Gaussian mixture data
X_gmm = torch.randn(100, 2)

print(f"Initial device: {next(gmm_hmm.parameters()).device}")

# Move to target device
gmm_hmm.to(device)
X_gmm_device = X_gmm.to(device)

print(f"After .to({device}): {next(gmm_hmm.parameters()).device}")

# Quick fit
gmm_hmm.fit(X_gmm_device, lengths=[50, 50], max_iter=5, n_init=1, verbose=False)
print(f"Training completed on {device}")


## 6. Save and Load Model Across Devices

You can save a model trained on one device and load it on another.
